In [9]:
import pickle
import numpy as np
import pandas as pd
import time
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

In [25]:
tqdm.pandas()

In [2]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

#使用pickle 加载词向量，速度更快，所加载的预训练的词向量也要为pkl文件
def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr
#加载普通txt和vec预训练词向量
def load_embeddings_normal(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [34]:
#建议英文文本时，不要将所有文本转化为小写，这可能丢失许多信息
#根据word_index 建立index_word矩阵，如果当前word在词中找不到对应的向量，转化为小写
def build_matrix(word_index, path):
    embedding_index = load_embeddings_normal(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        unknown_words.append(word)
    return embedding_matrix, unknown_words

In [4]:
#找出词向量中在训练数据数据中的覆盖率
import operator 
def check_coverage(vocab,embedding_index):
    a={}
    oov={}
    k=0
    i=0
    for word in tqdm(vocab):
        try:
            a[word]=embedding_index[word]
            k+=vocab[word]
        except:
            oov[word]=vocab[word]
            i+=vocab[word]
            pass
    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [ ]:
# 建立词典，即统计训练文本中每个词出现的频率
def build_vocab(sentences,verbose=True):
    vocab={}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [10]:
tic=time.time()
#词向量地址
GLOVE_EMBEDDING_PATH='./embedding/glove.840B.300d.txt'
#加载词向量
glove_embedding=load_embeddings_normal(GLOVE_EMBEDDING_PATH)
print(f'loaded {len(glove_embedding)} word vectors in {time.time()-tic}s')


loaded 2196007 word vectors in 156.29678750038147s


In [11]:
#训练样本
train = pd.read_csv('data/train.csv')
#建立词典
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())))
#获取训练样本中未向量化的词
oov = check_coverage(vocab,glove_embedding)
oov[:10]


Found embeddings for 15.82% of vocab
Found embeddings for  89.63% of all text


[("isn't", 39964),
 ("That's", 37640),
 ("won't", 29397),
 ("he's", 24353),
 ("Trump's", 23453),
 ("aren't", 20528),
 ("wouldn't", 19544),
 ('Yes,', 19043),
 ('that,', 18283),
 ("wasn't", 18153)]

In [12]:
import string
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar + ' '
#字母，数字，加上拉丁字母
white_list += "'"

In [13]:
white_list

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ '"

In [14]:
#glove中的字符
glove_chars = ''.join([c for c in tqdm(glove_embedding) if len(c) == 1])
#glove中的字符，并且不在write_list中
glove_symbols = ''.join([c for c in glove_chars if not c in white_list])

glove_symbols

',.":)(-!?|;$&/[]>%=#*+\\•~@£·_{}©^®`<→°€™›♥←×§″′█½…“★”–●►−¢²¬░¡¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■▀¨▄♫☆¯♦¤▲¸¾⋅∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤¹≤‡√◄━⇒▶º≥╝♡◊。✈≡☺✔↵≈✓♣☎℃◦└‟～！○◆№♠▌✿▸⁄□❖✦．÷｜┃／￥╠↩✭▐☼µ☻┐├«∼┌℉☮฿≦♬✧〉－⌂✖･◕※‖◀‰\x97↺∆┘┬╬،⌘⊂ª＞〈⎙Å？☠⇐▫∗∈≠♀ƒ♔˚℗┗＊┼❀＆∩♂‿∑‣➜┛⇓☯⊖☀┳；∇⇑✰◇♯☞´↔┏｡◘∂✌♭┣┴┓✨ˈ˜❥┫℠✒［∫\x93≧］\x94∀♛\x96∨◎ˑ↻⅓⇩＜≫✩ˆ✪♕؟₤☛╮␊＋┈ɡ％╋▽⇨┻⊗￡।▂✯▇＿➤₂✞＝▷△◙▅✝ﾟ∧␉☭┊╯☾➔∴\x92▃↳＾׳➢╭➡＠⊙☢˝⅛∏„①๑∥❝☐▆╱⋙๏☁⇔▔\x91②➚◡╰٠♢˙۞✘✮☑⋆ℓⓘ❒☣✉⌊➠∣❑⅔◢ⓒ\x80〒∕▮⦿✫✚⋯♩☂ˌ❞‗܂☜‾✜╲∘⟩＼⟨·⅜✗♚∅ⓔ◣͡‛❦⑨③◠✄❄１∃␣≪｢≅◯☽２∎｣⁰❧̅ǡⒶ↘⚓▣˘∪⇢✍⊥＃⅝⎯↠۩☰◥⊆✽ﬁ⚡↪ở❁☹◼☃◤❏ⓢ⊱α➝̣✡∠｀▴┤Ȃ∝♏ⓐ✎;３④␤＇❣⅞✂✤ⓞ☪✴⌒˛♒＄ɪ✶▻Ⓔ◌◈۲Ʈ❚ʿ❂￦◉╜̃ν✱╖❉₃ⓡℝ٤↗❶ʡ۰ˇⓣ♻➽۶₁ʃ׀✲ʤ✬☉▉≒☥⌐♨✕ⓝ⊰❘＂⇧̵➪４▁β۱▏⊃ⓛ‚♰́✏⏑̶٩Ⓢー⩾日￠❍≃⋰♋ɿ､̂❋✳ⓤ╤▕⌣✸℮⁺▨⑤╨Ⓥ♈❃☝５✻⊇≻♘♞◂７✟⌠✠☚✥❊ƂⒸ⌈❅Ⓡ♧Ⓞɑλ۵▭❱Ⓣ∟☕♺∵⍝ⓑɔ✵✣ℤ年ℕ٭♆Ⓘⅆ∶⚜◞்✹Ǥȡ➥ᴥ↕ɂ̳∷✋➧∋̿ͧʘ┅⥤⬆ǀμ₄⋱ʔ☄↖⋮۔♌Ⓛ╕♓ـ⁴❯♍▋✺⭐６✾♊➣▿Ⓑ♉Ａ⏠◾▹⑥⩽в↦╥⍵⌋։➨и∮⇥ⓗⒹ⁻ʊ⎝⌥⌉◔◑ǂ✼♎ℂ♐╪ɨ⊚☒⇤θВⓜ⎠Ｏ◐ǰ⚠╞ﬂ◗⎕ⓨ☟Ｉⓟ♟❈↬ⓓ◻♮❙а♤∉؛⁂例Ⓝ־♑╫╓╳⬅☔πɒɹ߂☸ɐʻ┄╧ʌ׃８ʒ⎢❆⋄⚫̏☏➞͂␙Ⓤ◟Ƥʕ̊Ȥ⚐✙は↙̾ωΔ℘ﾞ✷⑦φ⍺❌⊢▵✅ｗ９ⓖ☨▰ʹ╡Ⓜ☤∽╘˹↨ȿ♙⬇♱⌡Ω⠀╛❕┉Ⓟ̀Ǩ♖ⓚ┆⑧⎜ǹ◜⚾⤴✇╟⎛☩➲➟ⓥⒽ⏝◃０₀╢月↯✆˃⍴❇⚽╒Ｃɻɤ̸♜☓Ｔ➳⇄γ☬⚑✐⁵δȭ⌃◅▢ｓȸ❐∊☈ⅇℜ॥σ⎮ȣ▩のτεＳு⊹‵␔☊➸̌☿⇉➊⊳╙⁶ⓦ⇣｛̄↝⎟ℳ▍❗ℑＭɾｍ״Γ΄▞◁⛄⇝⎪ˤ♁ｖ⇠☇✊位ℒạி｝๐⭕➘Ｂ❺ɸˡ⁀⑩ｃ⅕Ƽ۳☙❛₆ƪ❓⟲Ʒ⇀≲Ｐ❷١ⓕ⎥Ｄс\u06ddǥͤ₋̱̎♝≳▙Ｒʹ➭ℰ܀ʺȫⒼ⇛ˉ▊❸号⇗̷

In [15]:
#建立字符词典
jigsaw_chars = build_vocab(list(train["comment_text"]))
#训练文本中除标准字符的其他乱七八糟的字符
jigsaw_symbols = ''.join([c for c in jigsaw_chars if not c in white_list])

jigsaw_symbols 

'.,?!-;*"…:\n—()%#$&_/@＼・ω+🍕=”“[]^–>\r🐵\\°<😑~\xa0\ue014•≠\t™\uf818\uf04a\xadˈʊɒ😢🐶∞§{}·τα❤️☺ɡ\uf0e0😜😎👊\u200b\u200e😁|عدويهصقأناخلىبمغر😍💖¢→̶`💵❥━┣┫Е┗Ｏ►★👎😀😂\u202a\u202c🔥😄©―🏻💥ᴍʏʀɪᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ✔®\x96\x92●😋👏שלוםבי😱‼£\x81♥エンジ故障➤´\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘¹☕≈÷אעכח♡◐║▬💩′ɔː💯⛽€🚄🏼ஜ۩۞†😖ᴠ🚲‐μ✒➥😟😈═☆ˌ💪🙏🎯◄🌹😇💔½ʻ😡\x7f👌ἐπὶδηλήσειὲκἀίῃἴρξνʃ🙄✬ＳＵＰＥＲＨＩＴ😠\ufeff☻±\u2028😉😤⛺♍🙂µ\u3000تحكسة👮💙فزط😏º🍾🎉¾😞\u2008🏾😅😭👻😥😔😓🏽🎆✓◾🍻🍽🎶🌺🤔😪\x08‑؟🐰🐇🐱🙆．😨⬅🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷℅»ВулканПвоАН🐾🐕❣😆ה⋅🔗¿¬🚽歌舞伎🙈😴🏿🤗🇺🇸♫мυтѕＣＭ⤵🏆🎃β😩█▓▒░\u200a🌠🐟💫💰💎⇒эпрд\x95🖐🙅⛲🍰⭐🤐👆›🙌\u2002💛🙁👀🙊🙉¡₂₃\u2004❧▰ˢᵒʳʸ▔ᴼᴷᴺʷᵗʰᵉᵘ◞▀\x13🚬▂▃▄▅▆▇↙🤓\ue602😵άοόςέγὸ̄תמדףנרךצט😒͝″☹➡«🆕👅👥👄🔄🔤👉👤👶👲🔛🎓φ\uf0b7⅓„✋：\uf04c\x9f\x10成都¥😣⏺̲̅😌🤑́🌏😯ех😲∙‛Ἰᾶὁ💞🚓◇🔔📚✏🏀👐\u202d💤🍇\ue613小土豆🏡▷❔❓⁉❗\u202f👠¶》कर्मा🇹🇼🌸蔡英文🌞˚🎲レクサス😛˙外国人关系）Ссиб💋💀🎄💜🤢َِʿьыгя✨不是。ɑ\x80\x9c\x9d🗑\u2005💃📣👿༼つ◕༽😰ḷЗз▱ц￼🤣卖！温哥华议会下降％你失去所有的钱加拿大坏税骗子🐝¯ツ🎅\x85🍺آإشء−ﬂﬁ🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003₁²🚀🤴ʌʲш¼⁴⁄₄⌠чИОРФДЯМю♭ж✘😝🖑ὐύύ特殊作戦群╪щ💨圆明园ק▶ℐ☭✭🏈😺♪🌍⏏ệ🍔🐮🍁☔🍆🍑🌮🌯☠🤦\u200d♂𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺☃🍸🥂🗽🎇🎊🆘☎🤠👩✈🖒✌✰❆☙🚪天一家⚲\u2006⚭⚆⬭⬯⏖○‣⚓新年∎ℒ▪▙☏⅛✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼

In [16]:
#这样就可以找出glove词向量中未向量的字符，但是在训练样本中出现的字符，也就是要处理的字符
symbols_to_delete = ''.join([c for c in jigsaw_symbols if not c in glove_symbols])
#训练文本中要删除的字符
symbols_to_delete

'\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊

In [17]:
#训练文本中要保留的字符
symbols_to_isolate = ''.join([c for c in jigsaw_symbols if c in glove_symbols])
symbols_to_isolate

'.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'

In [18]:
#ord函数是将一个字符转化为对应的ASCII数值
#将要保留的字符转化为字典
isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
#将要删除的字符转化为字典
remove_dict = {ord(c):f'' for c in symbols_to_delete}

In [19]:
isolate_dict

{46: ' . ',
 44: ' , ',
 63: ' ? ',
 33: ' ! ',
 45: ' - ',
 59: ' ; ',
 42: ' * ',
 34: ' " ',
 8230: ' … ',
 58: ' : ',
 8212: ' — ',
 40: ' ( ',
 41: ' ) ',
 37: ' % ',
 35: ' # ',
 36: ' $ ',
 38: ' & ',
 95: ' _ ',
 47: ' / ',
 64: ' @ ',
 65340: ' ＼ ',
 12539: ' ・ ',
 969: ' ω ',
 43: ' + ',
 61: ' = ',
 8221: ' ” ',
 8220: ' “ ',
 91: ' [ ',
 93: ' ] ',
 94: ' ^ ',
 8211: ' – ',
 62: ' > ',
 92: ' \\ ',
 176: ' ° ',
 60: ' < ',
 126: ' ~ ',
 8226: ' • ',
 8800: ' ≠ ',
 8482: ' ™ ',
 712: ' ˈ ',
 650: ' ʊ ',
 594: ' ɒ ',
 8734: ' ∞ ',
 167: ' § ',
 123: ' { ',
 125: ' } ',
 183: ' · ',
 964: ' τ ',
 945: ' α ',
 10084: ' ❤ ',
 9786: ' ☺ ',
 609: ' ɡ ',
 124: ' | ',
 162: ' ¢ ',
 8594: ' → ',
 822: ' ̶ ',
 96: ' ` ',
 10085: ' ❥ ',
 9473: ' ━ ',
 9507: ' ┣ ',
 9515: ' ┫ ',
 9495: ' ┗ ',
 65327: ' Ｏ ',
 9658: ' ► ',
 9733: ' ★ ',
 169: ' © ',
 8213: ' ― ',
 618: ' ɪ ',
 10004: ' ✔ ',
 174: ' ® ',
 150: ' \x96 ',
 146: ' \x92 ',
 9679: ' ● ',
 163: ' £ ',
 9829: ' ♥ ',
 10148: ' ➤ '

In [20]:
remove_dict

{10: '',
 127829: '',
 13: '',
 128053: '',
 128529: '',
 160: '',
 57364: '',
 9: '',
 63512: '',
 61514: '',
 173: '',
 128546: '',
 128054: '',
 65039: '',
 61664: '',
 128540: '',
 128526: '',
 128074: '',
 8203: '',
 8206: '',
 128513: '',
 1593: '',
 1583: '',
 1608: '',
 1610: '',
 1607: '',
 1589: '',
 1602: '',
 1571: '',
 1606: '',
 1575: '',
 1582: '',
 1604: '',
 1609: '',
 1576: '',
 1605: '',
 1594: '',
 1585: '',
 128525: '',
 128150: '',
 128181: '',
 1045: '',
 128078: '',
 128512: '',
 128514: '',
 8234: '',
 8236: '',
 128293: '',
 128516: '',
 127995: '',
 128165: '',
 7437: '',
 655: '',
 640: '',
 7431: '',
 628: '',
 7429: '',
 7439: '',
 7424: '',
 7435: '',
 668: '',
 7452: '',
 671: '',
 7451: '',
 7428: '',
 7448: '',
 665: '',
 1171: '',
 7434: '',
 7457: '',
 610: '',
 128523: '',
 128079: '',
 1513: '',
 1500: '',
 1493: '',
 1501: '',
 1489: '',
 1497: '',
 128561: '',
 8252: '',
 129: '',
 12456: '',
 12531: '',
 12472: '',
 25925: '',
 38556: '',
 8201:

In [21]:
def handle_punctuation(x):
    #translate 将根据字典进行替换，将文本中出现的key替换为value值
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

In [26]:
#处理数据
train['comment_text'] = train['comment_text'].progress_apply(lambda x:handle_punctuation(x))

In [27]:
#再次转化为字典
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())))
#检验
oov = check_coverage(vocab,glove_embedding)
oov[:10]


Found embeddings for 47.09% of vocab
Found embeddings for  98.68% of all text


[("isn't", 41947),
 ("That's", 38119),
 ("won't", 30974),
 ("he's", 25010),
 ("Trump's", 24059),
 ("aren't", 21489),
 ("wouldn't", 20066),
 ("wasn't", 18932),
 ("they're", 17834),
 ("there's", 15511)]

In [29]:
#处理缩写
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
def handle_contractions(x):
    x = tokenizer.tokenize(x)
    x = ' '.join(x)
    return x
train['comment_text'] = train['comment_text'].progress_apply(lambda x:handle_contractions(x))

In [30]:
#再次转化为字典，验证
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embedding)
oov[:10]

Found embeddings for 52.43% of vocab
Found embeddings for  99.58% of all text


[('tRump', 2521),
 ("gov't", 2237),
 ('Brexit', 1729),
 ('theglobeandmail', 1350),
 ("'the", 1302),
 ('Drumpf', 1183),
 ('deplorables', 989),
 ("'The", 843),
 ('SB91', 776),
 ('theguardian', 734)]

In [31]:
#处理开头为“'”的词，清除
def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x
train['comment_text'] = train['comment_text'].progress_apply(lambda x:fix_quote(x.split()))
#再次生成验证
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embedding)
oov[:50]

Found embeddings for 54.52% of vocab
Found embeddings for  99.66% of all text


[('tRump', 2522),
 ("gov't", 2237),
 ('Brexit', 1732),
 ('theglobeandmail', 1350),
 ('Drumpf', 1183),
 ('deplorables', 1023),
 ('SB91', 779),
 ('theguardian', 734),
 ("Gov't", 715),
 ('Trumpcare', 566),
 ('Trumpism', 543),
 ('bigly', 475),
 ('Klastri', 449),
 ("y'all", 396),
 ('Auwe', 386),
 ('2gTbpnsWATCH', 353),
 ('Trumpian', 350),
 ('Trumpsters', 340),
 ('Vinis', 321),
 ('Saullie', 298),
 ('shibai', 293),
 ('Koncerned', 287),
 ('SJWs', 281),
 ('TFWs', 276),
 ('RangerMC', 271),
 ('civilbeat', 269),
 ('klastri', 251),
 ('BCLibs', 248),
 ('Trudope', 242),
 ('garycrum', 242),
 ('Daesh', 241),
 ("Qur'an", 240),
 ('wiliki', 230),
 ('gofundme', 225),
 ('OBAMAcare', 222),
 ('cashapp24', 221),
 ('Donkel', 220),
 ('Finicum', 220),
 ('Trumpkins', 219),
 ('Cheetolini', 215),
 ('brotherIn', 214),
 ('11e7', 211),
 ('Beyak', 210),
 ('Trudeaus', 210),
 ('dailycaller', 207),
 ('Layla4', 205),
 ('Tridentinus', 203),
 ('Ontariowe', 202),
 ('washingtontimes', 200),
 ('Zupta', 196)]

In [35]:
#采用keras的text 工具包
from keras.preprocessing import text, sequence
#定义多少个词当作特征
max_features = 400000
tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False)
tokenizer.fit_on_texts(list(train['comment_text']) )
glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

n unknown words (glove):  152723
